In [1]:
import os
try:
	os.chdir(os.path.join(os.getcwd(), '.'))
	print(os.getcwd())
except:
	pass

d:\git\2nd-ML100Days


 # [作業目標]
 - 使用 Day 12 剛學到的方法, 對較完整的資料生成離散化特徵
 - 觀察上述離散化特徵, 對於目標值的預測有沒有幫助

 # [作業重點]
 - 仿照 Day 12 的語法, 將年齡資料 ('DAYS_BIRTH' 除以 365) 離散化
 - 繪製上述的 "離散化標籤" 與目標值 ('TARGET') 的長條圖

In [2]:
# 載入需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sb
warnings.filterwarnings('ignore')
# 設定 data_path
dir_data = './data/'


ModuleNotFoundError: No module named 'seaborn'

 ### 之前做過的處理

In [3]:
# 讀取資料檔
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)
app_train.shape



NameError: name 'dir_data' is not defined

In [4]:
# 將只有兩種值的類別型欄位, 做 Label Encoder, 計算相關係數時讓這些欄位可以被包含在內
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder, 以加入相關係數檢查
            app_train[col] = le.fit_transform(app_train[col])            
print(app_train.shape)
app_train.head()



ModuleNotFoundError: No module named 'sklearn'

In [5]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])


NameError: name 'app_train' is not defined

 ## 練習時間
 參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [6]:
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], 10)
age_data_discreted = age_data['YEARS_BINNED'].value_counts().sort_index()
age_groups  = age_data.groupby('YEARS_BINNED').mean()


NameError: name 'app_train' is not defined

In [7]:
plt.style.use('ggplot')
plt.figure(figsize = (10, 6))
px = age_data_discreted.index
py = age_groups['TARGET']
ax = sb.barplot(px, py)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
ax.set(xlabel="YEARS_BINNED")

NameError: name 'age_data_discreted' is not defined